In [1]:
# -*- coding: utf-8 -*-
import argparse
import copy
import json
import os
import pickle
import random
import sys
import re
import numpy as np
import openai
from typing import List, Dict, Any
import ast
from p3_utils import *
from tqdm import tqdm

#GPT_MODEL = "gpt-4-turbo-preview"  # [gpt-4-turbo-preview]
GPT_MODEL = "gpt-3.5-turbo"  # [gpt-3.5-turbo-preview]

setOpenAi(keyid =4)

0

In [2]:
def cot_solve(question):
    cot_prompt = """
    You will be given a programming puzzle.
    Your task is to find an input for the following function that will make the function return True. 
    Let's think step by step"""
    
    input_q = [
        {
            "role": "system",
            "content": cot_prompt
        },
        {
            "role": "user",
            "content": question
        }
    ]
    
    procedure = askChatGPT(input_q, model = GPT_MODEL, temperature=1)
    ans_procedure = {"role": "assistant", "content": procedure}
    input_q.append(ans_procedure)
    #final_q = {"role": "user", "content": "So, what is the input? You can either give the input or write a solution function in Python. Make sure the answer you return in the format 'def function_name(): return value'. Make sure NO explanation!"}
    final_q = {"role": "user", "content": """So, what is the input? Please only give the input in the format of a string and just give the answer without any additional explanation or clarification, no prefix or suffix.

    For example if the input should be x = 5, then you should only give the answer as 5 and not x = 5.
    For example, if the the input is list = [1,2,3], then you should only give the answer as [1,2,3] and not list = [1,2,3].
                """}
    input_q.append(final_q)
    final_answer = askChatGPT(input_q, model = GPT_MODEL, temperature=1)
    return final_answer

    

In [3]:
puzzles = []


with open("/Users/natehu/Desktop/Tsinghua Research/TaDe/dataset_gen_using_cot/p3_100_test_questions.json", "r") as f:
    puzzles = json.load(f)

In [14]:
id = 33

question = puzzles[id]["sat"]


In [15]:
solution = cot_solve(question)

In [16]:
solution

'-5.770237138115334'

In [17]:
converted_result = convert_to_type(puzzles[id]['ans_type'], solution)

In [18]:
exec(question)
result = sat(converted_result)

In [19]:
result

False

In [20]:
def calculate_price(number_of_token, type, model):
    if model == "gpt-3.5-turbo":
        if type == "input":
            per_token = 0.5 / 1000000
            return number_of_token * per_token
        elif type == "output":
            per_token = 1.5 / 1000000
            return number_of_token * per_token
    elif model == "gpt-4-turbo-preview":
        if type == "input":
            per_token = 10 / 1000000
            return number_of_token * per_token
        elif type == "output":
            per_token = 30 / 1000000
            return number_of_token * per_token

In [21]:
calculate_price(22606, "output", "gpt-3.5-turbo") + calculate_price(337062, "input", "gpt-3.5-turbo")

0.20243999999999998